In [1]:
# %load my_imports.py
import pandas as pd
import seaborn as sns
import pylab
from glob import glob
import os

params = {'legend.fontsize': 'x-large',
#           'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
          'axes.labelweight': 'bold',
         'axes.titlesize':'x-large',
          'axes.titleweight': 'bold',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/roee/workenv/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['pylab']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
fns = glob('/Users/roee/work/heritage/HHP_release3/*')

In [3]:
fns

['/Users/roee/work/heritage/HHP_release3/Claims.csv',
 '/Users/roee/work/heritage/HHP_release3/DaysInHospital_Y2.csv',
 '/Users/roee/work/heritage/HHP_release3/DaysInHospital_Y3.csv',
 '/Users/roee/work/heritage/HHP_release3/DrugCount.csv',
 '/Users/roee/work/heritage/HHP_release3/LabCount.csv',
 '/Users/roee/work/heritage/HHP_release3/Lookup PrimaryConditionGroup.csv',
 '/Users/roee/work/heritage/HHP_release3/Lookup ProcedureGroup.csv',
 '/Users/roee/work/heritage/HHP_release3/Members.csv',
 '/Users/roee/work/heritage/HHP_release3/Target.csv']

In [4]:
dfs = {os.path.basename(fn)[:-4]: pd.read_csv(fn) for fn in fns}

In [5]:
for name, df in dfs.iteritems():
    print name
    print df.shape
    print ', '.join(df.columns)
    print

Members
(113000, 3)
MemberID, AgeAtFirstClaim, Sex

LabCount
(361484, 4)
MemberID, Year, DSFS, LabCount

Lookup PrimaryConditionGroup
(45, 2)
PrimaryConditionGroup, Description

DrugCount
(818241, 4)
MemberID, Year, DSFS, DrugCount

Claims
(2668990, 14)
MemberID, ProviderID, Vendor, PCP, Year, Specialty, PlaceSvc, PayDelay, LengthOfStay, DSFS, PrimaryConditionGroup, CharlsonIndex, ProcedureGroup, SupLOS

Lookup ProcedureGroup
(17, 2)
ProcedureGroup, Description

DaysInHospital_Y3
(71435, 3)
MemberID, ClaimsTruncated, DaysInHospital

DaysInHospital_Y2
(76038, 3)
MemberID, ClaimsTruncated, DaysInHospital

Target
(70942, 3)
MemberID, ClaimsTruncated, DaysInHospital



In [6]:
dfs['DaysInHospital_Y3'].DaysInHospital.value_counts().sort_index()

0     60706
1      4464
2      2182
3      1429
4       842
5       528
6       287
7       218
8       143
9       115
10      103
11       65
12       62
13       50
14       23
15      218
Name: DaysInHospital, dtype: int64

In [7]:
x = dfs['DaysInHospital_Y3'].DaysInHospital
(x > 0).value_counts()

False    60706
True     10729
Name: DaysInHospital, dtype: int64

In [8]:
x = dfs['DaysInHospital_Y2'].DaysInHospital
(x > 0).value_counts()

False    64269
True     11769
Name: DaysInHospital, dtype: int64

In [9]:
x = set(dfs['DaysInHospital_Y2'].MemberID)
y = set(dfs['DaysInHospital_Y3'].MemberID)
z = set(dfs['Target'].MemberID)
print len(x), len(y), len(z)
print len(x.intersection(y)), len(x.intersection(z)), len(y.intersection(z))
print len(x.intersection(y).intersection(z))

76038 71435 70942
51966 43237 49682
39470


# Preparing the data

In [6]:
all_members = dfs['Claims'].MemberID.unique()
len(all_members)

113000

In [5]:
all_dfs = {}

### 1. Cohort flags

In [14]:
x = dfs['Claims']
y = pd.DataFrame(index=all_members)
y.index.name = 'MemberID'
y.loc[x[x.Year == 'Y1'].MemberID.unique(), 'is_in_y1'] = 1
y.loc[x[x.Year == 'Y2'].MemberID.unique(), 'is_in_y2'] = 1
y.loc[x[x.Year == 'Y3'].MemberID.unique(), 'is_in_y3'] = 1
y = y.fillna(0)
y['is_in_y1_y2'] = 1 * ((y['is_in_y1'] == 1) & (y['is_in_y2'] == 1))
df_cohort_flags = y
all_dfs['cohort_flags'] = y
y.head()

,is_in_y1,is_in_y2,is_in_y3,is_in_y1_y2
MemberID,,,,
42286978,1.0,0.0,0.0,0
97903248,1.0,1.0,1.0,1
2759427,1.0,1.0,1.0,1
73570559,1.0,1.0,1.0,1
11837054,1.0,1.0,1.0,1


### 2. Claims Truncated

In [44]:
x1 = dfs['DaysInHospital_Y2'].set_index('MemberID').ClaimsTruncated
x1.name = 'ClaimsTruncatedY1'
x2 = dfs['DaysInHospital_Y3'].set_index('MemberID').ClaimsTruncated
x2.name = 'ClaimsTruncatedY2'
df_claims_truncated = pd.concat([x1, x2], axis=1)
all_dfs['claims_truncated'] = df_claims_truncated

### 3. Age and Sex

In [16]:
x = dfs['Members'].set_index('MemberID')
x.AgeAtFirstClaim.value_counts()

40-49    16111
70-79    14514
50-59    13329
60-69    12622
30-39    12435
10-19    11319
0-9      10791
20-29     8505
80+       7621
Name: AgeAtFirstClaim, dtype: int64

In [17]:
def foo(c):
    if c == '80+':
        return 'age_85'
    a, b = c.split('-')
    return 'age_{}'.format(int((int(a) + int(b) + 1) / 2))
df_age = pd.get_dummies(x.AgeAtFirstClaim)
df_age = df_age.rename(columns={c: foo(c) for c in df_age.columns})
df_age['age_miss'] = (~df_age.any(axis=1)).astype('int')
all_dfs['age'] = df_age
df_age.head()

,age_5,age_15,age_25,age_35,age_45,age_55,age_65,age_75,age_85,age_miss
MemberID,,,,,,,,,,
14723353,0,0,0,0,0,0,0,1,0,0
75706636,0,0,0,0,0,0,0,1,0,0
17320609,0,0,0,0,0,0,0,1,0,0
69690888,0,0,0,0,1,0,0,0,0,0
33004608,1,0,0,0,0,0,0,0,0,0


In [18]:
df_age.age_miss.value_counts()

0    107247
1      5753
Name: age_miss, dtype: int64

In [19]:
df_sex = pd.get_dummies(x.Sex)
df_sex = df_sex.rename(columns={c: 'sex_' + c for c in df_sex.columns})
df_sex['sex_miss'] = (~df_sex.any(axis=1)).astype('int')
all_dfs['sex'] = df_sex
df_sex.head()

,sex_F,sex_M,sex_miss
MemberID,,,
14723353,0,1,0
75706636,0,1,0
17320609,0,1,0
69690888,0,1,0
33004608,0,1,0


### 4. Numbers of claims, providers, etc.

In [20]:
x = dfs['Claims']
y = pd.DataFrame(index=all_members)
for year in ['Y1', 'Y2']:
    g = x[x.Year == year].groupby('MemberID')
    for column in ['ProviderID', 'Vendor', 'PCP', 'Specialty', 'PlaceSvc', 'PrimaryConditionGroup', 'ProcedureGroup']:
        y['n_{}_{}'.format(column, year)] = g[column].nunique()
y.head()

,n_ProviderID_Y1,n_Vendor_Y1,n_PCP_Y1,n_Specialty_Y1,n_PlaceSvc_Y1,n_PrimaryConditionGroup_Y1,n_ProcedureGroup_Y1,n_ProviderID_Y2,n_Vendor_Y2,n_PCP_Y2,n_Specialty_Y2,n_PlaceSvc_Y2,n_PrimaryConditionGroup_Y2,n_ProcedureGroup_Y2
42286978,8.0,8.0,1.0,5.0,3.0,8.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97903248,3.0,3.0,1.0,3.0,2.0,4.0,3.0,10.0,9.0,1.0,6.0,5.0,7.0,6.0
2759427,2.0,2.0,1.0,2.0,2.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
73570559,7.0,6.0,1.0,5.0,3.0,4.0,6.0,3.0,3.0,1.0,3.0,2.0,2.0,4.0
11837054,6.0,4.0,1.0,4.0,2.0,7.0,6.0,7.0,6.0,1.0,6.0,3.0,7.0,5.0


In [21]:
df_numbers = y
all_dfs['counts'] = y

### 5. Length of stay

In [22]:
los_map = {'1 day': 1,
           '2 days': 2,
           '3 days': 3,
           '4 days': 4,
           '5 days': 5,
           '6 days': 6,
           '1- 2 weeks': 11,
           '2- 4 weeks': 21,
           '4- 8 weeks': 42,
           '26+ weeks': 180}

In [23]:
x.head()

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,NaN,8- 9 months,NEUMENT,0,MED,0
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,NaN,7- 8 months,NEUMENT,1-2,EM,0
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,NaN,0- 1 month,METAB3,0,EM,0
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,NaN,5- 6 months,METAB3,1-2,SCS,0
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,NaN,4- 5 months,FXDISLC,1-2,EM,0


In [24]:
x = dfs['Claims']
y = pd.DataFrame(index=all_members)
x['los_int'] = x.LengthOfStay.map(los_map)
x['los_unknown'] = (x.los_int.isnull()) & (x.SupLOS == 0)
x['los_suppressed'] = (x.los_int.isnull()) & (x.SupLOS == 1)
x['los_known'] = (~x.los_int.isnull())
for year in ['Y1', 'Y2']:
    x_this_year = x[x.Year == year]
    g_known = x_this_year[~x_this_year.los_int.isnull()].groupby('MemberID').los_int
    y['LOS_max_' + year] = g_known.max()
    y['LOS_min_' + year] = g_known.min()
    y['LOS_std_' + year] = g_known.std()
    y['LOS_mean_' + year] = g_known.mean()

    g = x_this_year.groupby('MemberID')
    y['LOS_TOT_UNKNOWN_' + year] = g.los_unknown.sum()
    y['LOS_TOT_KNOWN_' + year] = g.los_known.sum()
    y['LOS_TOT_SUPPRESSED_' + year] = g.los_suppressed.sum()

In [25]:
df_los = y
all_dfs['los'] = df_los

### 6. Charlson and days since first service

In [26]:
x.DSFS.value_counts()

0- 1 month      707721
1- 2 months     247343
2- 3 months     225216
3- 4 months     212214
5- 6 months     192000
4- 5 months     189001
6- 7 months     180662
7- 8 months     175191
8- 9 months     171878
9-10 months     151527
10-11 months    116328
11-12 months     47139
Name: DSFS, dtype: int64

In [27]:
x['dsfs_int'] = x[~x.DSFS.isnull()].DSFS.apply(lambda a:int(a.split('-')[0]) + 1)

In [28]:
x.dsfs_int.value_counts()

1.0     707721
2.0     247343
3.0     225216
4.0     212214
6.0     192000
5.0     189001
7.0     180662
8.0     175191
9.0     171878
10.0    151527
11.0    116328
12.0     47139
Name: dsfs_int, dtype: int64

In [29]:
x.CharlsonIndex.value_counts()

0      1356995
1-2    1256527
3-4      49479
5+        5989
Name: CharlsonIndex, dtype: int64

In [30]:
x['charlson_int'] = x.CharlsonIndex.map({'0': 0, '1-2': 2, '3-4': 4, '5+': 6})

In [31]:
x.charlson_int.value_counts()

0    1356995
2    1256527
4      49479
6       5989
Name: charlson_int, dtype: int64

In [32]:
y = pd.DataFrame(index=all_members)
for c in ['dsfs_int', 'charlson_int']:
    for year in ['Y1', 'Y2']:
        g = x[x.Year == year].groupby('MemberID')[c]
        cc = c.replace('_int', '')
        y['{}_{}_{}'.format(cc, 'max', year)] = g.max()
        y['{}_{}_{}'.format(cc, 'min', year)] = g.min()
        y['{}_{}_{}'.format(cc, 'avg', year)] = g.mean()
        y['{}_{}_{}'.format(cc, 'std', year)] = g.std()
        y['{}_{}_{}'.format(cc, 'range', year)] = g.max() - g.min()
df_dsfs = y
all_dfs['dsfs'] = y
y.head()

,dsfs_max_Y1,dsfs_min_Y1,dsfs_avg_Y1,dsfs_std_Y1,dsfs_range_Y1,dsfs_max_Y2,dsfs_min_Y2,dsfs_avg_Y2,dsfs_std_Y2,dsfs_range_Y2,charlson_max_Y1,charlson_min_Y1,charlson_avg_Y1,charlson_std_Y1,charlson_range_Y1,charlson_max_Y2,charlson_min_Y2,charlson_avg_Y2,charlson_std_Y2,charlson_range_Y2
42286978,11.0,1.0,6.666667,3.961940,10.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
97903248,9.0,1.0,4.666667,3.102126,8.0,11.0,1.0,6.000000,3.008759,10.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2759427,1.0,1.0,1.000000,0.000000,0.0,1.0,1.0,1.000000,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,0.0
73570559,9.0,1.0,4.350000,2.960708,8.0,9.0,1.0,6.800000,3.271085,8.0,2.0,0.0,1.5,0.888523,2.0,2.0,2.0,2.0,0.0,0.0
11837054,11.0,1.0,4.782609,2.645004,10.0,12.0,1.0,5.860465,2.578126,11.0,2.0,2.0,2.0,0.000000,0.0,2.0,2.0,2.0,0.0,0.0


### 7. One Hot PCG/PG/SP/PS

In [33]:
x = dfs['Claims'].set_index('MemberID')
x[['PrimaryConditionGroup', 'ProcedureGroup', 'PlaceSvc', 'Specialty']].nunique()

PrimaryConditionGroup    45
ProcedureGroup           17
PlaceSvc                  8
Specialty                12
dtype: int64

In [34]:
df_one_hots = \
    pd.get_dummies(x[['PrimaryConditionGroup', 'ProcedureGroup', 'PlaceSvc', 'Specialty']]).groupby(x.index).sum()

In [35]:
df_one_hots.head()

,PrimaryConditionGroup_AMI,PrimaryConditionGroup_APPCHOL,PrimaryConditionGroup_ARTHSPIN,PrimaryConditionGroup_CANCRA,PrimaryConditionGroup_CANCRB,PrimaryConditionGroup_CANCRM,PrimaryConditionGroup_CATAST,PrimaryConditionGroup_CHF,PrimaryConditionGroup_COPD,PrimaryConditionGroup_FLaELEC,...,Specialty_Emergency,Specialty_General Practice,Specialty_Internal,Specialty_Laboratory,Specialty_Obstetrics and Gynecology,Specialty_Other,Specialty_Pathology,Specialty_Pediatrics,Specialty_Rehabilitation,Specialty_Surgery
MemberID,,,,,,,,,,,,,,,,,,,,,
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
210,0,0,0,0,0,0,0,0,0,0,...,2,0,5,4,0,7,0,0,0,0
3197,0,0,0,0,0,0,0,0,2,0,...,2,0,10,2,0,0,0,6,0,0
3457,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3713,0,0,3,0,0,0,0,0,0,0,...,0,7,0,2,0,0,0,0,0,0


In [36]:
df_one_hots.shape

(113000, 82)

In [48]:
all_dfs['one_hots'] = df_one_hots

### 8. Lab Counts and Drug Count

In [63]:
x = dfs['DrugCount']
x.head()

,MemberID,Year,DSFS,DrugCount
0,48925661,Y2,9-10 months,7+
1,90764620,Y3,8- 9 months,3
2,61221204,Y1,2- 3 months,1
3,63628544,Y3,1- 2 months,1
4,46949606,Y2,10-11 months,3


In [87]:
y = pd.DataFrame(index=all_members)
for name in ['DrugCount', 'LabCount']:
    df = dfs[name].copy()
    
    # string to integer
    df['DSFS'] = df['DSFS'].apply(lambda a:int(a.split('-')[0]))
    
    # replace 7+ to 7 and 10+ to 10, and change to integer
    df[name] = df[name].str.replace('+', '').astype('int')
    
    for year in ['Y1', 'Y2']:
        g = df[df.Year == year].sort_values('DSFS').groupby('MemberID')[name]
        y['{}_{}_{}'.format(name, 'max', year)] = g.max()
        y['{}_{}_{}'.format(name, 'min', year)] = g.min()
        y['{}_{}_{}'.format(name, 'avg', year)] = g.mean()
        y['{}_{}_{}'.format(name, 'range', year)] = g.max() - g.min()
        y['{}_{}_{}'.format(name, 'months', year)] = g.count()
        y['{}_{}_{}'.format(name, 'velocity', year)] = g.apply(lambda a: a.iloc[-1] - a.iloc[0])
        y['{}_{}_{}'.format(name, 'is_null', year)] = y['{}_{}_{}'.format(name, 'max', year)].isnull()

all_dfs['drugs'] = y

In [88]:
y.head()

,DrugCount_max_Y1,DrugCount_min_Y1,DrugCount_avg_Y1,DrugCount_range_Y1,DrugCount_months_Y1,DrugCount_velocity_Y1,DrugCount_is_null_Y1,DrugCount_max_Y2,DrugCount_min_Y2,DrugCount_avg_Y2,...,LabCount_months_Y1,LabCount_velocity_Y1,LabCount_is_null_Y1,LabCount_max_Y2,LabCount_min_Y2,LabCount_avg_Y2,LabCount_range_Y2,LabCount_months_Y2,LabCount_velocity_Y2,LabCount_is_null_Y2
42286978,5.0,1.0,2.600000,4.0,10.0,0.0,False,NaN,NaN,NaN,...,1.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,True
97903248,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,...,5.0,-8.0,False,6.0,2.0,3.4,4.0,5.0,-4.0,False
2759427,6.0,2.0,3.700000,4.0,10.0,2.0,False,5.0,3.0,4.125000,...,1.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,True
73570559,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,...,3.0,-9.0,False,6.0,6.0,6.0,0.0,1.0,0.0,False
11837054,6.0,1.0,3.636364,5.0,11.0,2.0,False,5.0,2.0,3.416667,...,4.0,1.0,False,4.0,2.0,2.5,2.0,4.0,-2.0,False


### 9. Admission Risk

In [89]:
# winner's domain expert's admission risk value

df_admission_risk = pd.read_csv('/Users/roee/work/heritage/df_admission_risk.csv')
df_admission_risk = df_admission_risk.set_index('PCG')

df_admission_risk.head()

,AdmissionRiskG70 (age > 70),AdmissionRiskL70 (age < 70)
PCG,,
AMI,5,5
APPCHOL,1,1
ARTHSPIN,2,1
CANCRA,4,3
CANCRB,4,3


In [109]:
y = pd.DataFrame(index=all_members)

x = pd.merge(dfs['Claims'], df_admission_risk.reset_index(), 
             how='left', left_on='PrimaryConditionGroup', right_on='PCG')
x = x.set_index('MemberID')
x['age'] = dfs['Members'].set_index('MemberID').AgeAtFirstClaim

# taking age unknown as young
x['is_old'] = x.age.isin(['70-79', '80+'])

x['admission_risk'] = where(x.is_old, x['AdmissionRiskG70 (age > 70)'], x['AdmissionRiskL70 (age < 70)'])

g = x.groupby('MemberID')['admission_risk']
y['AdmissionRisk_max'] = g.max()
y['AdmissionRisk_ave'] = g.mean()

all_dfs['admission_risk'] = y
y.head()

,AdmissionRisk_max,AdmissionRisk_ave
42286978,3.0,1.416667
97903248,4.0,1.777778
2759427,2.0,1.111111
73570559,3.0,1.054545
11837054,5.0,1.705128


### 10. pcp/Vendor/Provider probabilities

- count distinct patients in Y2 for each PCP, vendor, provider.
- Take top 100 pcps, 200 vendors, 250 providers. Group all others in 'other'.
- calculate probability of the patients of each of those to be hospitalized.
- For each member, take the max prob for all their pcps/vendors/providers

In [23]:
total_days = pd.DataFrame(index=all_members)
total_days['tot'] = dfs['DaysInHospital_Y2'].set_index('MemberID').DaysInHospital
total_days['is_hosp'] = total_days.tot > 0
total_days.is_hosp.head()

42286978     True
97903248     True
2759427     False
73570559    False
11837054    False
Name: is_hosp, dtype: bool

In [28]:
probabilities = {}
for col, n_top in zip(['ProviderID', 'PCP', 'Vendor'], [250, 100, 200]):
    unique_members = dfs['Claims'][dfs['Claims'].Year == 'Y1'].groupby(col).MemberID.unique()
    unique_members.name = 'unique_members'

    counts = unique_members.apply(len)

    names = counts.sort_values(ascending=False)
    names.iloc[n_top:] = 'other'
    names.iloc[:n_top] = names.iloc[:n_top].index.values
    names.name = 'name'

    x = pd.concat((unique_members, names), axis=1)

    all_members_in_other = [member for list_of_members in x[x.name == 'other'].unique_members.values \
                            for member in list_of_members]
    all_members_in_other = set(all_members_in_other)
    len(all_members_in_other)

    x2 = x[x.name != 'other'].unique_members.apply(lambda a: total_days.is_hosp.loc[a].mean())
    x2.loc['other'] = total_days.is_hosp.loc[all_members_in_other].mean()
    x2.name = '{}_prob'.format(col)
    probabilities[col] = x2

In [29]:
probabilities['PCP'].head()

PCP
1303.0    0.141321
2136.0    0.127479
2448.0    0.474170
2469.0    0.178439
3394.0    0.178694
Name: PCP_prob, dtype: float64

##### And now for my next trick..

In [30]:
x = dfs['Claims'][['MemberID', 'ProviderID', 'Vendor', 'PCP']]
for k, v in probabilities.iteritems():
    x = x.set_index(k)
    
    # top N PCP/Vendor/Provider each with own probability
    x[v.name] = v
    
    # non top N = p(other)
    x = x.reset_index()
    claim_has_pcp = (~x[k].isnull())
    claim_not_in_top_n_pcps = (x[v.name].isnull())
    x.loc[claim_has_pcp & claim_not_in_top_n_pcps, v.name] = v.loc['other']

In [31]:
x.head()

,PCP,Vendor,ProviderID,MemberID,ProviderID_prob,Vendor_prob,PCP_prob
0,37796.0,172193.0,8013252.0,42286978,0.290076,0.292639,0.174658
1,5300.0,726296.0,3316066.0,97903248,0.184748,0.183639,0.165433
2,91972.0,140343.0,2997752.0,2759427,0.137230,0.107329,0.089273
3,70119.0,240043.0,7053364.0,73570559,0.195870,0.195870,0.165433
4,68968.0,496247.0,7557061.0,11837054,0.270588,0.277040,0.190416


In [32]:
x[x.ProviderID.isnull()].head()

,PCP,Vendor,ProviderID,MemberID,ProviderID_prob,Vendor_prob,PCP_prob
198,32724.0,NaN,NaN,45847458,NaN,NaN,0.139344
398,NaN,NaN,NaN,30614666,NaN,NaN,NaN
475,70853.0,NaN,NaN,90054573,NaN,NaN,0.165433
709,NaN,NaN,NaN,85492358,NaN,NaN,NaN
1005,27467.0,NaN,NaN,38061764,NaN,NaN,0.149725


In [36]:
df_probs = x[['ProviderID_prob', 'Vendor_prob', 'PCP_prob', 'MemberID']].groupby('MemberID').max(axis=1)

In [37]:
df_probs.head()

,ProviderID_prob,Vendor_prob,PCP_prob
MemberID,,,
4,0.184748,0.183639,0.184818
210,0.324277,0.342195,0.165433
3197,0.300000,0.263685,0.141119
3457,0.116071,0.118644,0.165433
3713,0.238938,0.217052,0.190416


In [38]:
# validation

In [39]:
probabilities['PCP'].loc[dfs['Claims'].set_index('MemberID').loc[210, 'PCP'].unique()]

PCP
37508.0         NaN
18880.0    0.149606
Name: PCP_prob, dtype: float64

In [40]:
probabilities['PCP'].loc[dfs['Claims'].set_index('MemberID').loc[3197, 'PCP'].unique()]

PCP
47016.0    0.141119
Name: PCP_prob, dtype: float64

In [41]:
probabilities['Vendor'].loc[dfs['Claims'].set_index('MemberID').loc[3197, 'Vendor'].unique()]

Vendor
122401.0    0.191720
844976.0    0.263685
35565.0     0.217052
240043.0    0.195870
835324.0    0.238342
Name: Vendor_prob, dtype: float64

In [42]:
all_dfs['df_probs'] = df_probs

### 11. Most Predictive Providers and Vendors

##### get top N vendors/Providers (there are too many to consider all of them in the Logistic Regression)

In [93]:
ys = {}
for col in ['ProviderID', 'Vendor']:
    unique_members = dfs['Claims'][dfs['Claims'].Year == 'Y1'].groupby(col).MemberID.unique()
    counts = unique_members.apply(len).sort_values(ascending=False)
    
    top_n = pd.Series(counts.iloc[:150].index.values)
    
    # take only claims with those vendors
    y = x[['MemberID', col]][x[col].isin(top_n)]
    assert y[col].nunique() == 150

    y = y.set_index('MemberID')
    y = pd.get_dummies(y, columns=[col]).groupby(y.index).max()
    
    # is hospitilied Y2 is the classification target
    y['target'] = (dfs['DaysInHospital_Y2'].set_index('MemberID').DaysInHospital > 0).astype('int')
    
    # take only those members who have data for hospitilization Y2
    y = y[~y.target.isnull()]
    ys[col] = y

In [94]:
ys['Vendor'].head()

,Vendor_2211.0,Vendor_5166.0,Vendor_9717.0,Vendor_26108.0,Vendor_31946.0,Vendor_35565.0,Vendor_41541.0,Vendor_50613.0,Vendor_62417.0,Vendor_64764.0,...,Vendor_924708.0,Vendor_931560.0,Vendor_934620.0,Vendor_955403.0,Vendor_969292.0,Vendor_972258.0,Vendor_979983.0,Vendor_982907.0,Vendor_988813.0,target
MemberID,,,,,,,,,,,,,,,,,,,,,
210,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3197,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3889,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
9063,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
11951,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1.0


In [96]:
y = ys['Vendor']
target = y.target
del y['target']

In [124]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [122]:
clf = LogisticRegression(penalty='l1', C=0.1)
clf.fit(y, target)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [123]:
sum(clf.coef_ != 0)

99

In [125]:
roc_auc_score(target, clf.predict_proba(y)[:, 1])

0.80245045368905177

##### Take 50 most predictive

# Concatenate all dfs

In [50]:
for name, df in all_dfs.iteritems():
    print name
    print df.shape
    print df.columns
    print

dsfs
(113000, 20)
Index([u'dsfs_max_Y1', u'dsfs_min_Y1', u'dsfs_avg_Y1', u'dsfs_std_Y1',
       u'dsfs_range_Y1', u'dsfs_max_Y2', u'dsfs_min_Y2', u'dsfs_avg_Y2',
       u'dsfs_std_Y2', u'dsfs_range_Y2', u'charlson_max_Y1',
       u'charlson_min_Y1', u'charlson_avg_Y1', u'charlson_std_Y1',
       u'charlson_range_Y1', u'charlson_max_Y2', u'charlson_min_Y2',
       u'charlson_avg_Y2', u'charlson_std_Y2', u'charlson_range_Y2'],
      dtype='object')

claims_truncated
(95507, 2)
Index([u'ClaimsTruncatedY1', u'ClaimsTruncatedY2'], dtype='object')

age
(113000, 10)
Index([u'age_5', u'age_15', u'age_25', u'age_35', u'age_45', u'age_55',
       u'age_65', u'age_75', u'age_85', u'age_miss'],
      dtype='object')

sex
(113000, 3)
Index([u'sex_F', u'sex_M', u'sex_miss'], dtype='object')

los
(113000, 14)
Index([u'LOS_max_Y1', u'LOS_min_Y1', u'LOS_std_Y1', u'LOS_mean_Y1',
       u'LOS_TOT_UNKNOWN_Y1', u'LOS_TOT_KNOWN_Y1', u'LOS_TOT_SUPPRESSED_Y1',
       u'LOS_max_Y2', u'LOS_min_Y2', u'LOS_std_Y2

TODO:
    - Go over everything and make sure i'm only using Y1 data for training (with DaysInHosp_Y2 as labels), and Y2 data for test (with DaysInHosp_Y3 as labels)